In [1]:
import pandas as pd
import numpy as np

In [7]:
import sys,os
print(os.getcwd())

/home/cayan/Documentos/Trabalhos/pd_imbalanced/notebooks


In [2]:
# Classifier Libraries
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV, GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

#import collections

# Other Libraries
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.ensemble import BalancedBaggingClassifier, BalancedRandomForestClassifier
from imblearn.metrics import classification_report_imbalanced
from imblearn.pipeline import Pipeline as imb_pipelineb
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline as imb_pipeline
from imblearn.under_sampling import TomekLinks

In [3]:
# GroupTimeSeriesSplit
from mlxtend.evaluate.time_series import (
    GroupTimeSeriesSplit,
    plot_splits,
    print_cv_info,
    print_split_info
)

In [4]:
from src.data.load_data import read_params, read_data

In [13]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:.2f}'.format

In [5]:
config = read_params(config_path="../params.yaml")

safra_config = config["proc_data_config"]["safra_config"]
shuffle_config = config["proc_data_config"]["shuffle_config"]

In [6]:
x_safra_train = read_data('../'  + safra_config["x_train_csv"])
y_safra_train = read_data('../'  + safra_config["y_train_csv"])

x_shuffle_train = read_data('../'  + shuffle_config["x_train_csv"])
y_shuffle_train = read_data('../'  + shuffle_config["y_train_csv"])

In [8]:
from src.models.parameters import (
    # safra
    safra_best_parameters_bbagg,
    safra_best_parameters_smote,
    safra_best_parameters_tomek,
    # shuffle
    shuffle_best_parameters_bbagg,
    shuffle_best_parameters_smote,
    shuffle_best_parameters_tomek
)

In [14]:
safra_split = {
        'safra_bbagg': safra_best_parameters_bbagg,
        'safra_smote': safra_best_parameters_smote,
        'safra_tomek': safra_best_parameters_tomek
}

shuffle_split = {
        'shuffle_bbagg': shuffle_best_parameters_bbagg,
        'shuffle_smote': shuffle_best_parameters_smote,
        'shuffle_tomek': shuffle_best_parameters_tomek
}

In [16]:
juntos = [safra_split, shuffle_split]

In [10]:
step_cen = {
        "pass": ("pass", "passthrough", [0]),
        "onehot_instrucao": ("cat", OneHotEncoder(), [1]),
        "std_all": ("num", StandardScaler(), [2, 3, 4, 5, 6]),
        "std_not_idade": ("num", StandardScaler(), [3, 4, 5, 6])
    }

preproc_i = ColumnTransformer(
            transformers=[step_cen["pass"], step_cen["onehot_instrucao"], step_cen["std_all"]],
            remainder="drop"
            )